# Reading and writing data -- _Tour of Beam_

So far we've learned some of the basic transforms like
[`Map`](https://beam.apache.org/documentation/transforms/python/elementwise/map),
[`FlatMap`](https://beam.apache.org/documentation/transforms/python/elementwise/flatmap),
[`Filter`](https://beam.apache.org/documentation/transforms/python/elementwise/filter),
[`Combine`](https://beam.apache.org/documentation/transforms/python/aggregation/combineglobally), and
[`GroupByKey`](https://beam.apache.org/documentation/transforms/python/aggregation/groupbykey).
These allow us to transform data in any way, but so far we've used
[`Create`](https://beam.apache.org/documentation/transforms/python/other/create)
to get data from an in-memory
[`iterable`](https://docs.python.org/3/glossary.html#term-iterable), like a `list`.

This works well for experimenting with small datasets. For larger datasets we can use `Source` transforms to read data and `Sink` transforms to write data.
If there are no built-in `Source` or `Sink` transforms, we can also easily create our custom I/O transforms.

Let's create some data files and see how we can read them in Beam.

In [1]:
%mkdir -p data

In [2]:
%%writefile data/my-text-file-1.txt
This is just a plain text file, UTF-8 strings are allowed 🎉.
Each line in the file is one element in the PCollection.

Writing data/my-text-file-1.txt


In [3]:
%%writefile data/my-text-file-2.txt
There are no guarantees on the order of the elements.
ฅ^•ﻌ•^ฅ

Writing data/my-text-file-2.txt


In [4]:
%%writefile data/penguins.csv
species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
1,0.5236363636363636,0.5714285714285713,0.3389830508474576,0.2222222222222222
1,0.6509090909090909,0.7619047619047619,0.4067796610169492,0.3333333333333333
2,0.509090909090909,0.011904761904761862,0.6610169491525424,0.5
2,0.6509090909090909,0.38095238095238104,0.9830508474576272,0.8333333333333334

Writing data/penguins.csv


# Reading from text files

We can use the
[`ReadFromText`](https://beam.apache.org/releases/pydoc/current/apache_beam.io.textio.html#apache_beam.io.textio.ReadFromText)
transform to read text files into `str` elements.

It takes a
[_glob pattern_](https://en.wikipedia.org/wiki/Glob_%28programming%29)
as an input, and reads all the files that match that pattern.
It returns one element for each line in the file.

For example, in the pattern `data/*.txt`, the `*` is a wildcard that matches anything. This pattern matches all the files in the `data/` directory with a `.txt` extension.

In [10]:
import apache_beam as beam

files = 'data/*.txt'

with beam.Pipeline ()as pipe:
    result = (
        pipe
        | "Read files" >> beam.io.ReadFromText(files)
        | "Print contents" >> beam.Map(print)
    )

There are no guarantees on the order of the elements.
ฅ^•ﻌ•^ฅ
This is just a plain text file, UTF-8 strings are allowed 🎉.
Each line in the file is one element in the PCollection.


### Writing to text files


In [ ]:
output_file_prefix = 'output/file'

text = [
          'Each element must be a string.',
          'It writes one element per line.',
          'There are no guarantees on the line order.',
          'The data might be written into multiple files.',
      ]

with beam.Pipeline() as pipe:
    (
        pipe
        | "Create file lines" >> beam.Create(text)
        | "Write to files" >> beam.io.WriteToText(
            output_file_prefix,
            file_name_suffix=".txt"
        )
    )


In [15]:
# Previewing the output
!head output/file*.txt

Each element must be a string.
It writes one element per line.
There are no guarantees on the line order.
The data might be written into multiple files.


### Reading from an `iterable`

In [22]:
import apache_beam as beam
from typing import Iterable

def counts(n: int) -> Iterable[int]:
    for i in range(n):
        yield i

# counts(4)
n = 5
with beam.Pipeline() as pipeline:
    (
        pipeline 
        | "Create inputs" >> beam.Create([n,6])
        | "Generate elements" >> beam.FlatMap(counts)
        | "Print elements" >> beam.Map(print)
    )

0
1
2
3
4
0
1
2
3
4
5


## Creating an input transform

In [25]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from typing import Iterable

@beam.ptransform_fn
@beam.typehints.with_input_types(beam.pvalue.PBegin)
@beam.typehints.with_output_types(int)
def Count(pbegin: beam.pvalue.PBegin, n: int) -> beam.PCollection[int]:
    def count(n: int) -> Iterable[int]:
        for i in range(n):
            yield i

    return (
        pbegin
        | "Create inputs" >> beam.Create([n])
        | "Generate element" >> beam.FlatMap(count)
    )


n = 5
options = PipelineOptions(flags=[], type_check_additional='all')
with beam.Pipeline(options=options) as pipeline:
    (
        pipeline
        | f"Count to {n}" >> Count(n)
        | "Print elements" >> beam.Map(print)
    )

0
1
2
3
4


### Example: Reading CSV files

In [26]:
import apache_beam as beam
from apache_beam.io.filesystems import FileSystems as beam_fs
from apache_beam.options.pipeline_options import PipelineOptions
import codecs
import csv
from typing import Dict, Iterable, List

@beam.ptransform_fn
@beam.typehints.with_input_types(beam.pvalue.PBegin)
@beam.typehints.with_output_types(Dict[str, str])
def ReadCsvFiles(pbegin: beam.pvalue.PBegin, file_patterns: List[str]) -> beam.PCollection[Dict[str, str]]:
    def expand_pattern(pattern: str) -> Iterable[str]:
        for match_result in beam_fs.match([pattern])[0].metadata_list:
            yield match_result.path

    def read_csv_lines(file_name: str) -> Iterable[Dict[str, str]]:
        with beam_fs.open(file_name) as f:
            # Beam reads files as bytes, but csv expects strings,
            # so we need to decode the bytes into utf-8 strings.
            for row in csv.DictReader(codecs.iterdecode(f, 'utf-8')):
                yield dict(row)

    return (
        pbegin
        | "Create file patterns" >> beam.Create(file_patterns)
        | "Expand file patterns" >> beam.FlatMap(expand_pattern)
        | "Read CSV lines" >> beam.FlatMap(read_csv_lines)
    )



input_patterns = ["data/*.csv"]
options = PipelineOptions(flags=[], type_check_additional='all')
with beam.Pipeline(options=options) as pipeline:
    (
        pipeline 
        | "Read CSV files" >> ReadCsvFiles(input_patterns)
        | "Print elements" >> beam.Map(print)
    )


{'species': '0', 'culmen_length_mm': '0.2545454545454545', 'culmen_depth_mm': '0.6666666666666666', 'flipper_length_mm': '0.15254237288135594', 'body_mass_g': '0.2916666666666667'}
{'species': '0', 'culmen_length_mm': '0.26909090909090905', 'culmen_depth_mm': '0.5119047619047618', 'flipper_length_mm': '0.23728813559322035', 'body_mass_g': '0.3055555555555556'}
{'species': '1', 'culmen_length_mm': '0.5236363636363636', 'culmen_depth_mm': '0.5714285714285713', 'flipper_length_mm': '0.3389830508474576', 'body_mass_g': '0.2222222222222222'}
{'species': '1', 'culmen_length_mm': '0.6509090909090909', 'culmen_depth_mm': '0.7619047619047619', 'flipper_length_mm': '0.4067796610169492', 'body_mass_g': '0.3333333333333333'}
{'species': '2', 'culmen_length_mm': '0.509090909090909', 'culmen_depth_mm': '0.011904761904761862', 'flipper_length_mm': '0.6610169491525424', 'body_mass_g': '0.5'}
{'species': '2', 'culmen_length_mm': '0.6509090909090909', 'culmen_depth_mm': '0.38095238095238104', 'flipper_l

- `DoFn`(a function object used with the `ParDo` transform), 
- `CombineFn` (a function object used with the `Combine` transform), and
- `WindowFn` (a function object used with the `Window` transform).



## Example: Reading from a SQLite database

In [27]:
#@title Creating the SQLite database
import sqlite3

database_file = "moon-phases.db" #@param {type:"string"}

with sqlite3.connect(database_file) as db:
  cursor = db.cursor()

  # Create the moon_phases table.
  cursor.execute('''
    CREATE TABLE IF NOT EXISTS moon_phases (
      id INTEGER PRIMARY KEY,
      phase_emoji TEXT NOT NULL,
      peak_datetime DATETIME NOT NULL,
      phase TEXT NOT NULL)''')

  # Truncate the table if it's already populated.
  cursor.execute('DELETE FROM moon_phases')

  # Insert some sample data.
  insert_moon_phase = 'INSERT INTO moon_phases(phase_emoji, peak_datetime, phase) VALUES(?, ?, ?)'
  cursor.execute(insert_moon_phase, ('🌕', '2017-12-03 15:47:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2017-12-10 07:51:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2017-12-18 06:30:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2017-12-26 09:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-02 02:24:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2018-01-08 22:25:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2018-01-17 02:17:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2018-01-24 22:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-31 13:27:00', 'Full Moon'))

  # Query for the data in the table to make sure it's populated.
  cursor.execute('SELECT * FROM moon_phases')
  for row in cursor.fetchall():
    print(row)

(1, '🌕', '2017-12-03 15:47:00', 'Full Moon')
(2, '🌗', '2017-12-10 07:51:00', 'Last Quarter')
(3, '🌑', '2017-12-18 06:30:00', 'New Moon')
(4, '🌓', '2017-12-26 09:20:00', 'First Quarter')
(5, '🌕', '2018-01-02 02:24:00', 'Full Moon')
(6, '🌗', '2018-01-08 22:25:00', 'Last Quarter')
(7, '🌑', '2018-01-17 02:17:00', 'New Moon')
(8, '🌓', '2018-01-24 22:20:00', 'First Quarter')
(9, '🌕', '2018-01-31 13:27:00', 'Full Moon')


In [28]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import sqlite3 as sq
from typing import Iterable, List, Tuple

class SQLiteSelect(beam.DoFn):
    def __init__(self, database_file: str):
        self.database_file = database_file
        self.connection = None

    def setup(self):
        self.connection = sq.connect(self.database_file)

    def process(self, query: Tuple[str, List[str]]) -> Iterable[Dict[str, str]]:
        table, columns = query
        cursor = self.connection.cursor()
        cursor.execute(f"SELECT {','.join(columns)} FROM {table}")
        for row in cursor.fetchall():
            yield dict(zip(columns, row))

    def teardown(self):
        self.connection.close()


@beam.ptransform_fn
@beam.typehints.with_input_types(beam.pvalue.PBegin)
@beam.typehints.with_output_types(Dict[str,str])
def SelectFromoSQLite(
    pbegin: beam.pvalue.PBegin,
    database_file: str,
    queries: List[Tuple[str, List[str]]],
) -> beam.PCollection[Dict[str, str]]:
    return (
        pbegin
        | "Create None" >> beam.Create(queries)
        | "SQLite SELECT" >> beam.ParDo(SQLiteSelect(database_file))
    )


queries = [
    # (table_name, [column1, column2, ...])
    ('moon_phases', ['phase_emoji', 'peak_datetime', 'phase']),
    ('moon_phases', ['phase_emoji', 'phase']),
]

options = PipelineOptions(flags=[], type_check_additional='all')
with beam.Pipeline(options=options) as pipeline:
    (
        pipeline
        | "Read from SQLite" >> SelectFromoSQLite(database_file, queries)
        | "Print rows" >> beam.Map(print)
    )

{'phase_emoji': '🌕', 'peak_datetime': '2017-12-03 15:47:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2017-12-10 07:51:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2017-12-18 06:30:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2017-12-26 09:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-02 02:24:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2018-01-08 22:25:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2018-01-17 02:17:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2018-01-24 22:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-31 13:27:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗',